<a href="https://colab.research.google.com/github/ram130849/Deep_Learning_Systems_Assignments/blob/main/TensorFLow/Sushant/DLS_Assignment_3_Part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 27.0 MB/s 


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_addons as tfa
import librosa
import librosa.display
import timeit
import os
from IPython.display import Audio
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import soundfile as sf
from tensorflow.keras.layers import Conv1D,Conv2D,MaxPooling1D,MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten
from keras.models import Model
import pickle
import tarfile

In [5]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [6]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
6.196400861000001
GPU (s):
0.08468183799999451
GPU speedup over CPU: 73x


#### 1) 1200 noisy speech signals (from trx0000.wav to trx1199.wav)
#### 2) 120 clean speech signals (10sentences per speaker for 12 speakers)
#### 3) Original Clean Speech - (from trs0000.wav to trs1199.wav)
#### 4) Noise Sources - (from trn0000.wav to trn1199.wav)
#### 5) Adding these clean and noise speeches we get trx speeches but since it is given in the folder, there is no need to do it

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
path_train = '/content/drive/MyDrive/DLS_Assignments/Assignment3_Data/timit-homework/tr/'
path_validation = '/content/drive/MyDrive/DLS_Assignments/Assignment3_Data/timit-homework/v/'
path_test = '/content/drive/MyDrive/DLS_Assignments/Assignment3_Data/timit-homework/te/'

In [9]:
!ls /content/drive/MyDrive/DLS_Assignments/Assignment3_Data/

timit-homework


In [10]:
def load_data(path):
  x=[]
  clean=[]
  noise=[]
  for file in os.listdir(path):
    if file.startswith('trx') or file.startswith('tex') or file.startswith('vx'):
      #if file not in x:
        x.append(file)

    elif file.startswith('trs') or file.startswith('tes') or file.startswith('vs'):
      #if file not in clean:
        clean.append(file)

    elif file.startswith('trn') or file.startswith('ten') or file.startswith('vn'):
      #if file not in noise:
        noise.append(file)
  
  x.sort()
  clean.sort()
  noise.sort()

  return x,clean,noise

### Loading the Train Data:

In [11]:
x_train, clean_train, noise_train = load_data(path_train)

In [12]:
print("The size of x_train is : ",len(x_train))
print("The size of clean_train is : ",len(clean_train))
print("The size of noise_train is : ",len(noise_train))

The size of x_train is :  1200
The size of clean_train is :  1200
The size of noise_train is :  1200


### Loading the Validation Data:

In [13]:
x_val, clean_val, noise_val = load_data(path_validation)

In [14]:
print("The size of x_val is : ",len(x_val))
print("The size of clean_val is : ",len(clean_val))
print("The size of noise_val is : ",len(noise_val))

The size of x_val is :  1200
The size of clean_val is :  1200
The size of noise_val is :  1200


### Loading the Test Data:

In [15]:
x_test, clean_test, noise_test = load_data(path_test)

In [16]:
print("The size of x_test is : ",len(x_test))
print("The size of clean_test is : ",len(clean_test))
print("The size of noise_test is : ",len(noise_test))

The size of x_test is :  400
The size of clean_test is :  0
The size of noise_test is :  0


In [22]:
def compute_stft(path,input):
  data_abs=[]
  for i in input:
    s, sr = librosa.load(path+i, sr=None)
    S = librosa.stft(s, n_fft=1024, hop_length=512)
    pad = np.zeros((513, 178))
    pad[:, :S.shape[1]] = S
    pad=pad.T
    pad_abs = np.abs(pad)
    data_abs.append(pad_abs)
  return data_abs

### Computing STFT for Training Data

In [23]:
x_train_stft = compute_stft(path_train,x_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys


In [31]:
print("Shape of x_train_stft is : ",x_train_stft[0].shape)
print(len(x_train_stft))

Shape of x_train_stft is :  (178, 513)
1200


In [24]:
clean_train_stft = compute_stft(path_train,clean_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys


In [32]:
print(len(clean_train_stft))

1200


In [25]:
noise_train_stft = compute_stft(path_train,noise_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys


### Computing STFT for Validation Data

In [26]:
x_val_stft = compute_stft(path_validation,x_val)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys


In [27]:
clean_val_stft = compute_stft(path_validation,clean_val)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys


In [28]:
noise_val_stft = compute_stft(path_validation,noise_val)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys


### Creating IBM Matrix for Training Set

In [34]:
ibm_train = [(clean_train_stft[i] > noise_train_stft[i]).astype(int) for i in range(len(clean_train_stft))]

In [35]:
ibm_train = np.stack(ibm_train)
noise_train_new = np.stack(noise_train_stft)

### Creating IBM Matrix for Validation Set

In [36]:
ibm_val = [(clean_val_stft[i] > noise_val_stft[i]).astype(int) for i in range(len(clean_val_stft))]

In [ ]:
ibm_val = np.stack(ibm_val)
noise_val_new = np.stack(noise_val_stft)